## 언어 모델(LLM)의 두 가지 유형

**LLM**: 문자열을 입력으로 받아 문자열을 반환하는 언어 모델입니다.

**ChatModels**: 메시지 목록을 입력으로 받아 메시지를 반환하는 언어 모델입니다.


---
LLM의 입력/출력은 문자열

하지만 ChatModels은 어떨까요? 입력은 ChatMessage의 목록이고 출력은 하나의 ChatMessage입니다. 


- ChatMessage에는 두 가지 필수 구성 요소:
    - content: 메시지의 내용입니다.
    - role: ChatMessage를 보낸 엔티티의 역할입니다.

---

- **예시**

```python
messages = [{"role": "user", "content": "hi"}]

result = openai.ChatCompletion.create(
    messages=messages,
    model="gpt-3.5-turbo",
    temperature=0
)

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
```


---

### LangChain은 서로 다른 역할을 쉽게 구분할 수 있도록 여러 객체를 제공

- HumanMessage: 사람/사용자로부터 오는 ChatMessage입니다.

- AIMessage: 인공지능/어시스턴트가 보내는 ChatMessage.

- SystemMessage: 시스템에서 보내는 챗메시지입니다.

- FunctionMessage: 함수 호출에서 오는 ChatMessage.


이러한 역할 중 어느 것도 적합하지 않은 경우 수동으로 역할을 지정할 수 있는 ChatMessage 클래스도 있습니다.  
이러한 다양한 메시지를 가장 효과적으로 사용하는 방법에 대한 자세한 내용은 프롬프트 가이드를 참조하세요.


predict: 문자열을 받아 문자열을 반환합니다.  
predict_messages: 메시지 목록을 받아 메시지를 반환합니다.  
이러한 다양한 유형의 모델과 다양한 유형의 입력으로 작업하는 방법을 살펴보겠습니다. 먼저 LLM과 ChatModel을 가져와 보겠습니다


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

In [2]:
import openai

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}]
)
print(completion.choices[0].message.content)

Hello! How can I assist you today?


In [4]:
result = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", messages=[{"role": "user", "content": "what are you doing?"}]
)

print(result["choices"][0]["message"])

{'role': 'assistant', 'content': 'I am an AI language model developed by OpenAI. I am here to answer questions and engage in conversation. How can I assist you today?'}


In [ ]:
# 위의 내용 연습하기

## Chat API : langchain

랭체인에서도 비슷하게 사용할 수 있다.  
좀 더 다양하고 편리한 기능들을 사용할 수 있다.


In [7]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [9]:
# 간단한 템플릿

from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template("나에게 {adjective} {content} 말해줘.")
prompt_template.format(adjective="재밌는", content="농담")

'나에게 재밌는 농담 말해줘.'

In [14]:
message = prompt_template.format(adjective="냉소적인", content="이야기")
message

str

### LangChain은 서로 다른 역할을 쉽게 구분할 수 있도록 여러 객체를 제공

- HumanMessage: 사람/사용자로부터 오는 ChatMessage입니다.

- AIMessage: 인공지능/어시스턴트가 보내는 ChatMessage.

- SystemMessage: 시스템에서 보내는 챗메시지입니다.

- FunctionMessage: 함수 호출에서 오는 ChatMessage.


In [16]:
# 채팅 프롬프트 템플릿

from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = template.format_messages(name="Bob", user_input="What is your name?")

messages

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, example=False),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, example=False),
 HumanMessage(content='What is your name?', additional_kwargs={}, example=False)]

In [22]:
res = chat(messages)

print(res.content)

content='My name is Bob. How can I assist you today?' additional_kwargs={} example=False


In [ ]:
# 문제: 프롬프트 템플릿을 가지고 여러가지 질문을 만들어보세요

# 1. 공부를 위한 채팅 프롬프트 만들기 질문하기

# 2. 여행계획 세우는 채팅 프롬프트 만들기 질문하기